## Import libraries 

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import predictor

## Load saved model

In [3]:
saved_model_dir= "./saved_model/1544257357"
predict_fn = predictor.from_saved_model(saved_model_dir)

INFO:tensorflow:Restoring parameters from ./saved_model/1544257357/variables/variables


In [4]:
def parse_func(s_example):
    features = {
                'workclass': tf.FixedLenFeature((), tf.string),
                'sex': tf.FixedLenFeature((), tf.string),
                'age': tf.FixedLenFeature((), tf.int64),
                'education': tf.FixedLenFeature((), tf.string),
                'education-num': tf.FixedLenFeature((), tf.int64),
                'marital-status': tf.FixedLenFeature((), tf.string),
                'occupation': tf.FixedLenFeature((), tf.string),
                'hours-per-week': tf.FixedLenFeature((), tf.int64),
                'label': tf.FixedLenFeature((), tf.string)
                 }
    return tf.parse_single_example(s_example, features=features)

In [5]:
data= tf.data.TFRecordDataset('census_test.tfrecord').batch(3)
next_batch= data.make_one_shot_iterator().get_next()
sess= tf.Session() 

## Read batches of serialized examples from `tfrecord` and feed them to predictor

In [9]:
examples = sess.run(next_batch)
predictions = predict_fn({'inputs':examples})

print(predictions['scores'],[sess.run(parse_func(example))['label'] for example in examples])

[[0.8598315  0.14016849]
 [0.19413778 0.8058622 ]
 [0.384946   0.6150541 ]] [b' <=50K.', b' >50K.', b' <=50K.']


In [17]:
predictions

{'scores': array([[0.64114726, 0.35885268],
        [0.44513756, 0.55486244],
        [0.67866397, 0.3213361 ]], dtype=float32),
 'classes': array([[b'0', b'1'],
        [b'0', b'1'],
        [b'0', b'1']], dtype=object)}

In [15]:
sess.close()

### Inspect saved model meta data with `saved_model_cli show`

In [3]:
from tensorflow.python.tools import saved_model_cli

In [4]:
!saved_model_cli show --dir saved_model/1544257357 --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "classification"
SignatureDef key: "predict"
SignatureDef key: "regression"
SignatureDef key: "serving_default"


### Use `saved_model_cli run` to make inference against saved model (`predict` Signature)
#### First, save the batch of examples to `.npy` file
#### Then, use `saved_model_cli` to run the model by reading inputs from `.npy` file.

In [ ]:
np.save('examples.npy',examples)

In [11]:
!saved_model_cli run --dir saved_model/1544257357 \
 --tag_set serve --signature_def predict \
 --inputs examples=examples.npy

2018-12-09 23:47:05.260278: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Result for output key class_ids:
[[0]
 [1]
 [1]]
Result for output key classes:
[[b'0']
 [b'1']
 [b'1']]
Result for output key logistic:
[[0.14016849]
 [0.8058622 ]
 [0.6150541 ]]
Result for output key logits:
[[-1.8138913]
 [ 1.4233446]
 [ 0.4686072]]
Result for output key probabilities:
[[0.8598315  0.14016849]
 [0.19413778 0.8058622 ]
 [0.384946   0.6150541 ]]


### Send inference request to the model serviced by tensorflow model server over REST api

In [13]:
import base64

encoded = np.array([base64.b64encode(example) for example in examples])
encoded

array([b'CoIDChEKBmZubHdndBIHGgUKA6S0BgoUCgVsYWJlbBILCgkKByA8PTUwSy4KDAoDYWdlEgUaAwoBNwoZCgllZHVjYXRpb24SDAoKCgggN3RoLTh0aAoXCg5ob3Vycy1wZXItd2VlaxIFGgMKAQoKEAoDc2V4EgkKBwoFIE1hbGUKHAoMcmVsYXRpb25zaGlwEgwKCgoIIEh1c2JhbmQKFQoMY2FwaXRhbC1nYWluEgUaAwoBAAokCg5uYXRpdmUtY291bnRyeRISChAKDiBVbml0ZWQtU3RhdGVzCh8KCm9jY3VwYXRpb24SEQoPCg0gQ3JhZnQtcmVwYWlyChYKDWVkdWNhdGlvbi1udW0SBRoDCgEECikKDm1hcml0YWwtc3RhdHVzEhcKFQoTIE1hcnJpZWQtY2l2LXNwb3VzZQoVCgxjYXBpdGFsLWxvc3MSBRoDCgEAChIKBHJhY2USCgoICgYgV2hpdGUKGQoJd29ya2NsYXNzEgwKCgoIIFByaXZhdGU=',
       b'CocDChAKA3NleBIJCgcKBSBNYWxlChwKDHJlbGF0aW9uc2hpcBIMCgoKCCBIdXNiYW5kChYKDGNhcGl0YWwtZ2FpbhIGGgQKApIyCiQKDm5hdGl2ZS1jb3VudHJ5EhIKEAoOIFVuaXRlZC1TdGF0ZXMKJAoKb2NjdXBhdGlvbhIWChQKEiBNYWNoaW5lLW9wLWluc3BjdAoWCg1lZHVjYXRpb24tbnVtEgUaAwoBCQopCg5tYXJpdGFsLXN0YXR1cxIXChUKEyBNYXJyaWVkLWNpdi1zcG91c2UKFQoMY2FwaXRhbC1sb3NzEgUaAwoBAAoSCgRyYWNlEgoKCAoGIFdoaXRlChkKCXdvcmtjbGFzcxIMCgoKCCBQcml2YXRlChEKBmZubHdndBIHGgUKA4ahCwoTCgVsYWJlbBIKCggKBiA+NTBLLgoMCgNhZ2USBRoDCgFBChk

In [14]:
encoded[0]

b'CoIDChEKBmZubHdndBIHGgUKA6S0BgoUCgVsYWJlbBILCgkKByA8PTUwSy4KDAoDYWdlEgUaAwoBNwoZCgllZHVjYXRpb24SDAoKCgggN3RoLTh0aAoXCg5ob3Vycy1wZXItd2VlaxIFGgMKAQoKEAoDc2V4EgkKBwoFIE1hbGUKHAoMcmVsYXRpb25zaGlwEgwKCgoIIEh1c2JhbmQKFQoMY2FwaXRhbC1nYWluEgUaAwoBAAokCg5uYXRpdmUtY291bnRyeRISChAKDiBVbml0ZWQtU3RhdGVzCh8KCm9jY3VwYXRpb24SEQoPCg0gQ3JhZnQtcmVwYWlyChYKDWVkdWNhdGlvbi1udW0SBRoDCgEECikKDm1hcml0YWwtc3RhdHVzEhcKFQoTIE1hcnJpZWQtY2l2LXNwb3VzZQoVCgxjYXBpdGFsLWxvc3MSBRoDCgEAChIKBHJhY2USCgoICgYgV2hpdGUKGQoJd29ya2NsYXNzEgwKCgoIIFByaXZhdGU='

#### `post_msg` , a file whose JSON formated contents are part of inference request to be sent over REST api.  The binary bytes (ex: serialization of structured data)  have to be Base64 encoded. 

In [12]:
!cat post_msg

{
"signature_name": "predict",
"inputs":[
{"b64":"CoEDChQKBWxhYmVsEgsKCQoHIDw9NTBLLgoMCgNhZ2USBRoDCgEZChYKCWVkdWNhdGlvbhIJCgcKBSAxMXRoChcKDmhvdXJzLXBlci13ZWVrEgUaAwoBKAoQCgNzZXgSCQoHCgUgTWFsZQoeCgxyZWxhdGlvbnNoaXASDgoMCgogT3duLWNoaWxkChUKDGNhcGl0YWwtZ2FpbhIFGgMKAQAKJAoObmF0aXZlLWNvdW50cnkSEgoQCg4gVW5pdGVkLVN0YXRlcwokCgpvY2N1cGF0aW9uEhYKFAoSIE1hY2hpbmUtb3AtaW5zcGN0ChYKDWVkdWNhdGlvbi1udW0SBRoDCgEHCiQKDm1hcml0YWwtc3RhdHVzEhIKEAoOIE5ldmVyLW1hcnJpZWQKFQoMY2FwaXRhbC1sb3NzEgUaAwoBAAoSCgRyYWNlEgoKCAoGIEJsYWNrChkKCXdvcmtjbGFzcxIMCgoKCCBQcml2YXRlChEKBmZubHdndBIHGgUKA/LrDQ=="},
{"b64":"CoUDChwKDHJlbGF0aW9uc2hpcBIMCgoKCCBIdXNiYW5kChUKDGNhcGl0YWwtZ2FpbhIFGgMKAQAKJAoObmF0aXZlLWNvdW50cnkSEgoQCg4gVW5pdGVkLVN0YXRlcwoiCgpvY2N1cGF0aW9uEhQKEgoQIEZhcm1pbmctZmlzaGluZwoWCg1lZHVjYXRpb24tbnVtEgUaAwoBCQopCg5tYXJpdGFsLXN0YXR1cxIXChUKEyBNYXJyaWVkLWNpdi1zcG91c2UKFQoMY2FwaXRhbC1sb3NzEgUaAwoBAAoSCgRyYWNlEgoKCAoGIFdoaXRlChkKCXdvcmtjbGFzcxIMCgoKCCBQcml2YXRlChEKBmZubHdndBIHGgUKA9a9BQoUCgVsYWJlbBILCgkKByA8PTUwSy4KDAoDYW

#### Use `curl` to send inference request to the model serviced by tensorflow model server over REST api

In [21]:
!curl -d @post_msg -X POST http://localhost:8501/v1/models/census:predict

{
    "outputs": {
        "class_ids": [
            [
                0
            ],
            [
                1
            ],
            [
                0
            ]
        ],
        "probabilities": [
            [
                0.641147,
                0.358853
            ],
            [
                0.445138,
                0.554862
            ],
            [
                0.678664,
                0.321336
            ]
        ],
        "classes": [
            [
                "0"
            ],
            [
                "1"
            ],
            [
                "0"
            ]
        ],
        "logits": [
            [
                -0.580347
            ],
            [
                0.220337
            ],
            [
                -0.747639
            ]
        ],
        "logistic": [
            [
                0.358853
            ],
            [
                0.554862
            ],
            [
              